In [15]:
from sklearn import datasets,svm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [4]:
df= datasets.load_iris()

In [6]:
data_Set= pd.DataFrame(df['data'],columns=df.feature_names)

In [10]:
data_Set['target']= df.target

In [13]:
data_Set['flower_name']= data_Set.target.apply(lambda x:df.target_names[x])

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
df.data, df.target, test_size=0.25)

In [32]:
grid_search= svm.SVC(gamma="auto",kernel='linear',C=10)
grid_search.fit(X_train,y_train)
grid_search.score(X_test,y_test)

0.9210526315789473

### cross value score

In [39]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),df.data,df.target,cv=5)


array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [40]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),df.data,df.target,cv=5)


array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

# Grid search CV 

In [51]:
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [5,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(df.data, df.target)
clf.cv_results_

{'mean_fit_time': array([0.00312433, 0.        , 0.        , 0.00443058, 0.        ,
        0.00312939]),
 'std_fit_time': array([0.00624866, 0.        , 0.        , 0.00614321, 0.        ,
        0.00625877]),
 'mean_score_time': array([0., 0., 0., 0., 0., 0.]),
 'std_score_time': array([0., 0., 0., 0., 0., 0.]),
 'param_C': masked_array(data=[5, 5, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 5, 'kernel': 'rbf'},
  {'C': 5, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],
 'split0_test_score': array([0.96666667, 1.        , 0.96666667, 1.        , 0.96666667,
        1.        ]),
 

### want to show that data in proper form(table) 

In [52]:
df1= pd.DataFrame(clf.cv_results_)

In [53]:
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003124,0.006249,0.0,0.0,5,rbf,"{'C': 5, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000000,0.000000,0.0,0.0,5,linear,"{'C': 5, 'kernel': 'linear'}",1.000000,1.0,0.933333,0.966667,1.0,0.980000,0.026667,1
2,0.000000,0.000000,0.0,0.0,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.004431,0.006143,0.0,0.0,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000000,0.000000,0.0,0.0,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.003129,0.006259,0.0,0.0,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [57]:
df1[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,5,rbf,0.980000
1,5,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [58]:
clf.best_score_

0.9800000000000001

In [59]:
clf.best_params_

{'C': 5, 'kernel': 'rbf'}

### want to pick best different values of C 

In [65]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(df.data, df.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

# the outcome results of mean test score is same. now I check them by models

,param_C,param_kernel,mean_test_score
0,10,linear,0.973333
1,10,rbf,0.980000


### in which I try best param by different models

In [67]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [69]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(df.data, df.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
